Codice per allenamento con TRIPLET LOSS dei 2 modelli creati.

In [ ]:
# associazione tra i numeri delle classi e nome delle classi dell'abito

# sort con natural interpretation, using NaturalSort
with open('/content/drive/MyDrive/Computer Vision Project/Code/Retrieval/ass.json') as f:
  ass = json.load(f)

ass

Impostazione dei parametri:

In [1]:
import os
import json
import time
import torch
import random
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from natsort import natsorted
from sklearn.model_selection import train_test_split
from torchsummary import summary
from PIL import Image
import cv2

RANDOMSTATE=0

In [2]:
# impostare random seed
torch.manual_seed(2020)
np.random.seed(2020)
random.seed(2020)

In [3]:
if torch.cuda.is_available():
  device=torch.device('cuda:0')
else:
  device=torch.device('cpu')
print(device)

if device.type == "cuda":
    print(torch.cuda.get_device_name())

cuda:0
Tesla T4


Import dati:

In [4]:
embedding_dims = 2 # non usato
batch_size = 32
epochs = 10

In [5]:
PATH= '/content/drive/MyDrive/Computer Vision Project/Code/Retrieval/tot_df_TL.csv'

In [6]:
df = pd.read_csv(PATH)
df.head()

,anchor,POS,NEG
0,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...
1,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...
2,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...
3,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...
4,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...


In [7]:
df, df_cut= train_test_split(df, test_size=0.4) #, shuffle=False
df.shape

(9219, 3)

In [8]:
def prepare_data(DF):
    DF, _ = train_test_split(DF, test_size=0.2, random_state=RANDOMSTATE)
    train_part, testDF = train_test_split(DF, test_size=0.2, random_state=RANDOMSTATE)
    #trainDF, validateDF = train_test_split(train_part, test_size=0.4, random_state=RANDOMSTATE)

    return train_part, testDF

In [9]:
# 1° autoencoder
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        self.encoder = nn.Sequential(# in- (N,3,512,512)
            
            nn.Conv2d(in_channels=3, 
                      out_channels=16, 
                      kernel_size=(3,3), 
                      stride=3, 
                      padding=1),  # (32,16,171,171)
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # (N,16,85,85)
            
            nn.Conv2d(in_channels=16, 
                      out_channels=8, 
                      kernel_size=(3,3), 
                      stride=2, 
                      padding=1),  # (N,8,43,43)
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # (N,8,42,42)
        )
        self.decoder = nn.Sequential(
            
            nn.ConvTranspose2d(in_channels = 8, 
                               out_channels=16, 
                               kernel_size=(3,3), 
                               stride=2),  # (N,16,85,85)
            nn.ReLU(True),
 
            nn.ConvTranspose2d(in_channels=16, 
                               out_channels=8, 
                               kernel_size=(5,5), 
                               stride=3, 
                               padding=1),  # (N,8,255,255)
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=8, 
                               out_channels=3, 
                               kernel_size=(6,6), 
                               stride=2, 
                               padding=1),  # (N,3,512,512)
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [9]:
# 2° autoencoder
class Conv_layers(nn.Module):
  def __init__(self):
    super(Conv_layers, self).__init__()

    self.encoder= nn.Sequential(# N,3,512,512   cambiare la dimensione
                             
        nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, padding=0, stride=1),
        nn.ReLU(True),
        nn.MaxPool2d(2, stride=2),

        nn.Conv2d(in_channels=8, out_channels=32, kernel_size=3, stride=2),
        nn.ReLU(True),
        nn.MaxPool2d(2, stride=2),

        nn.Conv2d(in_channels=32, out_channels=128, kernel_size=3, stride=1),
        nn.ReLU(True),
        nn.MaxPool2d(2, stride=2),

        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1),
        nn.ReLU(True), 
        nn.MaxPool2d(2, stride=2),
        
    )
  
    self.decoder= nn.Sequential(
        nn.ConvTranspose2d(in_channels = 256, 
                               out_channels=128, 
                               kernel_size=3, 
                               stride=3, padding=2, dilation=3),
        nn.ConvTranspose2d(in_channels = 128, 
                               out_channels=32, 
                               kernel_size=(3,3), 
                               stride=2),  # (N,16,85,85)
        nn.ReLU(True),
 
        nn.ConvTranspose2d(in_channels=32, 
                               out_channels=8, 
                               kernel_size=(5,5), 
                               stride=3, 
                               padding=1),  # (N,8,255,255)
        nn.ReLU(True),

        nn.ConvTranspose2d(in_channels=8, 
                               out_channels=3, 
                               kernel_size=(6,6), 
                               stride=2, 
                               padding=1),  # (N,3,512,512)
        nn.Tanh()
        
    )
     
  
  def forward(self, x):
    x= self.encoder(x)
    x= self.decoder(x)
    return x

In [10]:
class TrainDataset(Dataset):
  def __init__(self, df):
    self.df= df
    self.transform= transforms.Compose([
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                        transforms.Resize((512, 512))
    ])

    # per permettere di leggere (con l'item) ogni riga e quindi percorso delle varie immagini
    self.anchor= df['anchor'].values
    self.pos= df['POS'].values
    self.neg= df['NEG'].values

  def __len__(self):
    return len(self.df.index)

  def __getitem__(self, item):
    # trasformo - apro - (self.anchor[item])
    anchor = self.transform(Image.open(self.anchor[item])) 
    positive= self.transform(Image.open(self.pos[item]))
    negative= self.transform(Image.open(self.neg[item]))

    return anchor, positive, negative


class TestDataset(Dataset):
  def __init__(self, df):
    self.df= df
    self.anchor= df['anchor'].values
    self.transform= transforms.Compose([
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                        transforms.Resize((512, 512))
    ])

  def __len__(self):
    return len(self.df.index)
  
  def __getitem__(self, item):
    anchor= self.transform(Image.open(self.anchor[item]))
    return anchor

Carico i dati per l'allenamento:

In [11]:
train, test=prepare_data(DF=df)

In [12]:
# ATTENZIONE
# aggiungo ulteriore split per velocizzare le prove per capire il funzionamento!!! 

train, _ = train_test_split(train, test_size=0.5, random_state=RANDOMSTATE)

In [13]:
train= train.reset_index()
train= train.drop(columns='index')

In [14]:
train.index

RangeIndex(start=0, stop=2950, step=1)

In [15]:
train.head()

,anchor,POS,NEG
0,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...
1,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...
2,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...
3,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...
4,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...,/content/drive/MyDrive/Computer Vision Project...


In [16]:
train_set= TrainDataset(train)

train_loader = DataLoader(train_set,
                         batch_size=batch_size,
                         shuffle=True,
                         num_workers=0, pin_memory=True)


In [17]:
test_set= TrainDataset(test)

test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

weights:

In [18]:
def init_weights(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_normal_(m.weight)

Model:

In [19]:
model = ConvAutoencoder()
model.apply(init_weights)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.TripletMarginWithDistanceLoss(distance_function=lambda x, y: 1.0 - nn.functional.cosine_similarity(x, y))
# distance_function=nn.PairwiseDistance()

#criterion = nn.TripletMarginWithDistanceLoss(distance_function= nn.CosineSimilarity())

Training:

In [ ]:
'''
def save_checkpoint(state, filename):
    print ("=> Saving a new best")
    torch.save(state, filename)
'''

In [20]:
model.train()

for epoch in tqdm(range(epochs), desc="Epochs"):
    running_loss = []
    for step, (anchor_img, positive_img, negative_img) in enumerate(tqdm(train_loader, desc="Training", leave=False)):
        anchor_img = anchor_img.to(device)
        positive_img = positive_img.to(device)
        negative_img = negative_img.to(device)
        
        optimizer.zero_grad()
        anchor_out = model(anchor_img)
        positive_out = model(positive_img)
        negative_out = model(negative_img)
          
        loss = criterion(anchor_out, positive_out, negative_out)

        loss.backward()
        optimizer.step()
        
        running_loss.append(loss.cpu().detach().numpy())
    print("Epoch: {}/{} - Loss: {:.4f}".format(epoch+1, epochs, np.mean(running_loss)))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 1/10 - Loss: 0.9276


Epoch: 2/10 - Loss: 0.7926


Epoch: 3/10 - Loss: 0.7652


Epoch: 4/10 - Loss: 0.8064


...

Epoch: 6/10 - Loss: 0.7513


Epoch: 7/10 - Loss: 0.7978


Epoch: 8/10 - Loss: 0.7528


Epoch: 9/10 - Loss: 0.7360


Epoch: 10/10 - Loss: 0.7357



In [21]:
# SAVE the model:
percorso1='clothes_TL1_ae1_10ep.pt' # 10ep
torch.save(model.state_dict(), percorso1)

# torch.save({"model_state_dict": model.state_dict(), "optimzier_state_dict": optimizer.state_dict()}, "trained_model.pth")

In [22]:
percorso2='clothes_TL2model_ae1_10ep.pt' # 10ep
torch.save(model, percorso2)

alternativa:

In [ ]:
best_loss= np.inf

model.train()
scaler = torch.cuda.amp.GradScaler()
for epoch in tqdm(range(epochs), desc="Epochs"):
    running_loss = []
    for step, (anchor_img, positive_img, negative_img) in enumerate(tqdm(train_loader, desc="Training", leave=False)):
        anchor_img = anchor_img.to(device)
        positive_img = positive_img.to(device)
        negative_img = negative_img.to(device)
        
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
          anchor_out = model(anchor_img)
          positive_out = model(positive_img)
          negative_out = model(negative_img)
          
          loss = criterion(anchor_out, positive_out, negative_out)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        
        running_loss.append(loss.cpu().detach().numpy())
    print("Epoch: {}/{} - Loss: {:.4f}".format(epoch+1, epochs, np.mean(running_loss)))
    ###
    '''
    if np.mean(running_loss) < best_loss:
      save_checkpoint(state={'state_dict': model.state_dict()},
                      filename='ckpt_epoch_{}.pt'.format(epoch))
    '''


# Parte di valutazione

In [ ]:
train_results = []
labels = []

model.eval()
with torch.no_grad():
    for img, _, _, label in tqdm(train_loader):
        train_results.append(model(img.to(device)).cpu().numpy())
        labels.append(label)
        
train_results = np.concatenate(train_results)
labels = np.concatenate(labels)
train_results.shape

In [ ]:
plt.figure(figsize=(15, 10), facecolor="azure")
for label in np.unique(labels):
    tmp = train_results[labels==label]
    plt.scatter(tmp[:, 0], tmp[:, 1], label=label)

plt.legend()
plt.show()

In [ ]:
tree = XGBClassifier(seed=2020)
tree.fit(train_results, labels)

In [ ]:
test_results = []
test_labels = []

model.eval()
with torch.no_grad():
    for img in tqdm(test_loader):
        test_results.append(model(img.to(device)).cpu().numpy())
        
test_results = np.concatenate(test_results)

plt.figure(figsize=(15, 10), facecolor="azure")
plt.scatter(test_results[:, 0], test_results[:, 1], label=label)

test_results.shape

In [ ]:
tree.predict(test_results)